In [1]:
!pip install dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 6.2 MB 800 kB/s 


In [64]:
# Ref: https://cnvrg.io/graph-neural-networks/

In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.


In [3]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [10]:
g = dataset[0]
# num_class = g.num_classes
# get node feature
feat = g.ndata['feat']
# get data split
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
# get labels
label = g.ndata['label']
print(feat.shape, train_mask.shape, val_mask.shape, test_mask.shape, label.shape)

torch.Size([2708, 1433]) torch.Size([2708]) torch.Size([2708]) torch.Size([2708]) torch.Size([2708])


In [14]:
g.ndata['feat'].shape[1]

1433

In [62]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, temp_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)

        return h

# Create the model with given dimensions
# model = GCN(g.ndata['feat'].shape[1], 160, dataset.num_classes)

In [63]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(1000):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 160, 160, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.114 (best 0.114), test acc: 0.091 (best 0.091)
In epoch 5, loss: 1.844, val acc: 0.698 (best 0.698), test acc: 0.724 (best 0.724)
In epoch 10, loss: 1.683, val acc: 0.720 (best 0.722), test acc: 0.756 (best 0.764)
In epoch 15, loss: 1.469, val acc: 0.738 (best 0.738), test acc: 0.754 (best 0.754)
In epoch 20, loss: 1.215, val acc: 0.766 (best 0.766), test acc: 0.782 (best 0.782)
In epoch 25, loss: 0.946, val acc: 0.778 (best 0.778), test acc: 0.784 (best 0.787)
In epoch 30, loss: 0.696, val acc: 0.782 (best 0.782), test acc: 0.792 (best 0.792)
In epoch 35, loss: 0.492, val acc: 0.786 (best 0.786), test acc: 0.792 (best 0.793)
In epoch 40, loss: 0.342, val acc: 0.786 (best 0.786), test acc: 0.797 (best 0.793)
In epoch 45, loss: 0.239, val acc: 0.782 (best 0.788), test acc: 0.806 (best 0.802)
In epoch 50, loss: 0.169, val acc: 0.784 (best 0.788), test acc: 0.802 (best 0.802)
In epoch 55, loss: 0.122, val acc: 0.788 (best 0.788), test acc: 0.799 (best 0